In [1]:
import pandas as pd

In [2]:
elus_2020 = pd.read_parquet('/home/onyxia/work/Python-project/Data/elus-conseillers-municipaux-cm.parquet')
elus_2020.head()

,Code du département,Libellé du département,Code de la collectivité à statut particulier,Libellé de la collectivité à statut particulier,Code de la commune,Libellé de la commune,Nom de l'élu,Prénom de l'élu,Code sexe,Date de naissance,Code de la catégorie socio-professionnelle,Libellé de la catégorie socio-professionnelle,Date de début du mandat,Libellé de la fonction,Date de début de la fonction,Code nationalité
0,01,Ain,<NA>,None,01001,L'Abergement-Clémenciat,BEAUDET,Sylvie,F,1967-03-25,12,Agriculteur sur moyenne exploitation,2020-05-18,None,None,FR
1,01,Ain,<NA>,None,01001,L'Abergement-Clémenciat,BERAUD,Zélie,F,1980-03-26,43,Profession intermédiaire de la santé et du tra...,2020-05-18,None,None,FR
2,01,Ain,<NA>,None,01001,L'Abergement-Clémenciat,BOUILLOUX,Delphine,F,1977-02-08,38,Ingénieur et cadre technique d'entreprise,2020-05-18,None,None,FR
3,01,Ain,<NA>,None,01001,L'Abergement-Clémenciat,BOULON,Daniel,M,1951-04-03,74,Ancien cadre,2020-05-18,Maire,2020-05-26,FR
4,01,Ain,<NA>,None,01001,L'Abergement-Clémenciat,BUET,Roger,M,1952-04-21,71,Ancien agriculteur exploitant,2020-05-18,3ème adjoint au Maire,2020-05-26,FR
